## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
#!pip install mysql-connector
#!pip install mosestokenizer
import mysql.connector
import pandas as pd
import html
import os
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from string import punctuation
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
import nltk
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))
import regex as re
import glob
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/halledavis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
punctuation = set(punctuation) # speeds up comparison
tw_punct = punctuation - {"#"}

sw = set(nltk.corpus.stopwords.words('english'))

whitespace_pattern = re.compile(r"\s+")
hashtag_pattern = re.compile(r"^#[0-9a-zA-Z]+")

def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    counter = Counter()
    tokens.map(counter.update)
    freq_df = pd.DataFrame.from_dict(counter, orient='index', columns=['freq'])
    counter_df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
    num_tokens = sum(freq_df['freq'])
    num_unique_tokens = freq_df.shape[0]
    lexical_diversity = num_unique_tokens / num_tokens
    num_characters = sum((counter_df['index'].str.len()) * counter_df[0])

    if verbose :
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
        print(f"The top 5 most common words are")
        print(counter.most_common(5))

    return(0)

def remove_stop(tokens) :
    return [t for t in tokens if t.lower() not in stopwords]
 
def remove_punctuation(text, punct_set=tw_punct) : 
    return("".join([ch for ch in text if ch not in punct_set]))

def tokenize(text):
    return re.findall(r'\S+', text)

def prepare(text, pipeline) : 
    tokens = str(text)
    
    for transform in pipeline : 
        tokens = transform(tokens)
        
    return(tokens)

In [4]:
def clean(text):
    # convert html escapes like &amp; to characters.
    text = html.unescape(text)
    # tags like <tab>
    text = re.sub(r'<[^<>]*>', ' ', text)
    # markdown URLs like [Some text](https://....)
    text = re.sub(r'\[([^\[\]]*)\]\([^\(\)]*\)', r'\1', text)
    # text or code in brackets like [0]
    text = re.sub(r'\[[^\[\]]*\]', ' ', text)
    # standalone sequences of specials, matches &# but not #cool
    text = re.sub(r'(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)', ' ', text) # standalone sequences of hyphens like --- or ==
    text = re.sub(r'(?:^|\s)[\-=\+]{2,}(?:\s|$)', ' ', text)
    # sequences of white spaces
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

In [5]:
my_pipeline = [str.lower, remove_punctuation, tokenize, remove_stop]

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [6]:
conn = sqlite3.connect("2020_Conventions.db")
cursor = conn.cursor()

In [7]:
cursor.execute("""SELECT name FROM sqlite_schema
WHERE type='table'
ORDER BY name;""")

cursor.fetchall()

[('conventions',)]

In [43]:
data = pd.read_sql("""
                        SELECT *
                        FROM conventions;
                        """,
                        conn)
#print(data.head())

In [9]:
data["tokens"] = data["text"].apply(prepare,pipeline=my_pipeline)

In [10]:
data["cleantext"] = data["text"].apply(clean)
data["cleantext"] = data["cleantext"].apply(str.lower)
data["cleantext"] = data["cleantext"].apply(remove_punctuation)

In [11]:
data.head()

,party,night,speaker,speaker_count,time,text,text_len,file,tokens,cleantext
0,Democratic,4,Unknown,1,00:00,Skip to content The Company Careers Press Free...,127,www_rev_com_blog_transcripts2020-democratic-na...,"[skip, content, company, careers, press, freel...",skip to content the company careers press free...
1,Democratic,4,Speaker 1,1,00:33,I’m here by calling the full session of the 48...,41,www_rev_com_blog_transcripts2020-democratic-na...,"[i’m, calling, full, session, 48th, quadrennia...",i’m here by calling the full session of the 48...
2,Democratic,4,Speaker 2,1,00:59,"Every four years, we come together to reaffirm...",17,www_rev_com_blog_transcripts2020-democratic-na...,"[every, four, years, come, together, reaffirm,...",every four years we come together to reaffirm ...
3,Democratic,4,Kerry Washington,1,01:07,We fight for a more perfect union because we a...,28,www_rev_com_blog_transcripts2020-democratic-na...,"[fight, perfect, union, fighting, soul, countr...",we fight for a more perfect union because we a...
4,Democratic,4,Bernie Sanders,1,01:18,"We must come together to defeat Donald Trump, ...",22,www_rev_com_blog_transcripts2020-democratic-na...,"[must, come, together, defeat, donald, trump, ...",we must come together to defeat donald trump a...


In [12]:
#convention_data = []

#query_results = cursor.execute("""SELECT text, party
#                        FROM conventions;""")


#for row in query_results :
#    convention_data.append(row)

In [13]:
query_results = data[["cleantext", "party"]]

In [14]:
convention_data = query_results.values.tolist()

Let's look at some random entries and see if they look right. 

In [44]:
#random.choices(convention_data,k=10)

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [16]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2510 as features in the model.


In [17]:
my_pipeline2 = [str.lower, tokenize]

data["tokens"] = data["cleantext"].apply(prepare,pipeline=my_pipeline)

In [18]:
def conv_features(text,fw) :
    tokenfw = {}
    tokens = tokenize(text)
    for token in tokens:
        if token in fw :
            tokenfw[token] = True
        #    tokenfw.append(
        #    {
        #        token,
        #        True
        #    }
        #)
            
    return(tokenfw)

In [19]:
assert(len(feature_words)>0)
print("Test 1")
print(conv_features("donald is the president",feature_words))
print("\n Test 2")
print(conv_features("people are american in america",feature_words))
#assert(conv_features("donald is the president",feature_words)== {'donald':True,'president':True})
#assert(conv_features("people are american in america",feature_words)=={'america':True,'american':True,"people":True})

Test 1
{'donald': True, 'is': True, 'the': True, 'president': True}

 Test 2
{'people': True, 'are': True, 'american': True, 'in': True, 'america': True}


Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [20]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [45]:
#featuresets

In [22]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [23]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.444


In [24]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

##### Write a little prose here about what you see in the classifier. Anything odd or interesting?

Right off the bat, I notice the word "J." -- I would love to see this in context, but my guess is this is in reference to the name "Donald J. Trump". I think it's interesting because it shows that Republicans are more likely to add the middle initial when referencing President Trump, which makes sense because it's slightly more honorific. It also makes sense that "Lady" would be on their list because that are more likely to make reference to First Lady Melania Trump.

I notice that "great" and "first" are associated with Republicans, which makes sense because it's associated with kep slogans or policies that Republicans had at the time (America First and Make America Great Again). 

After that I just see key policy differences between Republicans and Democrats: enforcement, media, destroy, China, private, freedom, defund, and preserve, all seem related to Republican concerns or traditional philosophies. Alternatively, Democrats have "votes" and "climate", which related to climate change and protecting the right to vote.


## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [25]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [26]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [27]:
resultsdf = pd.DataFrame(results, columns=['author', 'party', 'text'])

In [28]:
text = []

for row in resultsdf["text"]:
    try:
        text.append(row.decode())
    except:
        text.append(row.encode())

In [29]:
resultsdf["text"] = text

In [30]:
resultsdf["tokens"] = resultsdf["text"].apply(prepare,pipeline=my_pipeline)

In [31]:
resultsdf["cleantext"] = resultsdf["text"].apply(clean)
resultsdf["cleantext"] = resultsdf["cleantext"].apply(str.lower)
resultsdf["cleantext"] = resultsdf["cleantext"].apply(remove_punctuation)


# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

In [32]:
resultsdf

,author,party,text,tokens,cleantext
0,Mo Brooks,Republican,"""Brooks Joins Alabama Delegation in Voting Aga...","[brooks, joins, alabama, delegation, voting, f...",brooks joins alabama delegation in voting agai...
1,Mo Brooks,Republican,"""Brooks: Senate Democrats Allowing President t...","[brooks, senate, democrats, allowing, presiden...",brooks senate democrats allowing president to ...
2,Mo Brooks,Republican,"""NASA on the Square"" event this Sat. 11AM – 4P...","[nasa, square, event, sat, 11am, –, 4pm, stop,...",nasa on the square event this sat 11am – 4pm s...
3,Mo Brooks,Republican,"""The trouble with Socialism is that eventually...","[trouble, socialism, eventually, run, peoples,...",the trouble with socialism is that eventually ...
4,Mo Brooks,Republican,"""The trouble with socialism is eventually you ...","[trouble, socialism, eventually, run, peoples,...",the trouble with socialism is eventually you r...
...,...,...,...,...,...
664651,David McKinley,Republican,We had a great time at the WVU Homecoming para...,"[great, time, wvu, homecoming, parade, yesterd...",we had a great time at the wvu homecoming para...
664652,David McKinley,Republican,We need more transparency in Washington #wvpol...,"[need, transparency, washington, #wvpol, https...",we need more transparency in washington #wvpol...
664653,David McKinley,Republican,We saw there is a double standard in DC and th...,"[saw, double, standard, dc, rules, simply, don...",we saw there is a double standard in dc and th...
664654,David McKinley,Republican,Wow! Huge crowd in Charleston at the @WVGOP vi...,"[wow, huge, crowd, charleston, wvgop, victory,...",wow huge crowd in charleston at the wvgop vict...


In [46]:
query_results = resultsdf[["cleantext", "party"]]
tweet_data = query_results.values.tolist()
#tweet_data

In [34]:
word_cutoff = 5

tokens = [w for t, p in tweet_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 51762 as features in the model.


In [35]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in tweet_data]
tweet_data = featuresets

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [36]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [37]:
classifier = nltk.NaiveBayesClassifier.train(tweet_data_sample)

In [38]:
for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(tweet)
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")

Here's our (cleaned) tweet: {'earlier': True, 'today': True, 'i': True, 'spoke': True, 'on': True, 'the': True, 'house': True, 'floor': True, 'abt': True, 'protecting': True, 'health': True, 'care': True, 'for': True, 'women': True, 'and': True, 'praised': True, 'their': True, 'work': True, 'central': True, 'coast': True}
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: {'go': True, 'tribe': True, '#rallytogether': True}
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: {'apparently': True, 'trump': True, 'thinks': True, 'its': True, 'just': True, 'too': True, 'easy': True, 'for': True, 'students': True, 'overwhelmed': True, 'by': True, 'the': True, 'crushing': True, 'burden': True, 'of': True, 'debt': True, 'to': True, 'pay': True, 'off': True, 'student': True, 'loans': True, '#trumpbudget': True}
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: {'we’re': True, 

In [39]:
print(nltk.classify.accuracy(classifier, tweet_data))

0.48471991526443753


In [40]:
classifier.show_most_informative_features(25)

Most Informative Features
                     and = True           Republ : Democr =      3.0 : 1.0
                    help = True           Republ : Democr =      3.0 : 1.0
                   their = True           Republ : Democr =      3.0 : 1.0
                    #kag = None           Democr : Republ =      1.9 : 1.0
                    been = None           Democr : Republ =      1.9 : 1.0
                    even = None           Democr : Republ =      1.9 : 1.0
            firefighters = None           Democr : Republ =      1.9 : 1.0
                   first = None           Democr : Republ =      1.9 : 1.0
                grateful = None           Democr : Republ =      1.9 : 1.0
                 greater = None           Democr : Republ =      1.9 : 1.0
                      it = None           Democr : Republ =      1.9 : 1.0
                    keep = None           Democr : Republ =      1.9 : 1.0
                   let’s = None           Democr : Republ =      1.9 : 1.0

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [41]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(tweet)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [42]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 1408, 'Democratic': 2870}),
             'Democratic': defaultdict(int,
                         {'Republican': 2303, 'Democratic': 3421})})

### Reflections

Looking at our most important features, I see that the indicativeness of our key words is a lot lower than for the congressional db data, perhaps due to the smaller sample size. Our accuracy on the dataset as a whole though is slightly higher -- 48%.

Our key words (aka most important features) don't seem to be as keenly associated as the congressional db was. I can't quite create the connection between the party and the keyword. 